In [1]:
from scripts.tools.issuetracker import IssuetrackerAPI
import re

api = IssuetrackerAPI('issue.pbsbiotech.com', 'nstarkweather', 'kookychemist')
issues = api.download_issues(project_id="pbssoftware")
filtered = [i for i in issues.values() if i.sprint_milestone.name == "3.0"]


magic = re.compile(r"[\+\*]+(3.0Web)?(FRS\d+[0-9\.]+)")
magic2 = re.compile(r"(3.0Web)?(FRS\d+[0-9\.]+)")

In [2]:
frs = set()
frs2 = set()

for v in filtered:
    d = v.description or ""
    l = magic.findall(d)
    for w, f in l:
        w = w or ""
        if "." not in f and v.assigned_to.name == "Christian Nilsson" and not w:
            w = "3.0Web"
        result = (v.id, w+f)
        frs.add(result)
    l = magic2.findall(d)
    for w, f in l:
        w = w or ""
        if "." not in f and v.assigned_to.name == "Christian Nilsson" and not w:
            w = "3.0Web"
        result = (v.id, w+f)
        frs2.add(result)
    
sorted(frs)

[(420, 'FRS420.0'),
 (977, 'FRS977.0'),
 (989, 'FRS989.0'),
 (1073, 'FRS1073.01'),
 (1073, 'FRS1073.02'),
 (1073, 'FRS1073.03'),
 (1073, 'FRS1073.11'),
 (1073, 'FRS1073.1304'),
 (1073, 'FRS1073.1305'),
 (1073, 'FRS1073.1306'),
 (1073, 'FRS1073.1307'),
 (1073, 'FRS1073.1308'),
 (1073, 'FRS1073.1309'),
 (1073, 'FRS1073.1310'),
 (1073, 'FRS1073.1311'),
 (1073, 'FRS1073.1312'),
 (1073, 'FRS1073.1313'),
 (1073, 'FRS1073.1314'),
 (1073, 'FRS1073.1315'),
 (1073, 'FRS1073.1316'),
 (1262, 'FRS1262.0'),
 (1263, 'FRS1263.0'),
 (1583, 'FRS1583.0'),
 (1583, 'FRS1583.1'),
 (1583, 'FRS1583.2'),
 (1583, 'FRS1583.3'),
 (1583, 'FRS1583.4'),
 (1754, 'FRS1754.0'),
 (1759, 'FRS1759.0'),
 (1845, 'FRS1845.0'),
 (1902, 'FRS1902.0'),
 (2115, 'FRS2115.0'),
 (2210, 'FRS2210.0'),
 (2303, 'FRS2303.0'),
 (2304, 'FRS2304.1'),
 (2304, 'FRS2304.10'),
 (2304, 'FRS2304.11'),
 (2304, 'FRS2304.12'),
 (2304, 'FRS2304.13'),
 (2304, 'FRS2304.14'),
 (2304, 'FRS2304.15'),
 (2304, 'FRS2304.16'),
 (2304, 'FRS2304.17'),
 (2304, '

In [3]:
from scripts.tools.issuetracker import IssuetrackerAPI
import re
try:
    cache
except NameError:
    cache = None
cache = cache or {}
magic = re.compile(r"[\+\*\s]*(3.0Web)?(FRS\d+[0-9\.]+)[\+\*: \-]*")

def get_issues():
    global cache
    if not cache:
        api = IssuetrackerAPI('issue.pbsbiotech.com', 'nstarkweather', 'kookychemist')
        iss = api.download_issues(project_id="pbssoftware")
        cache = iss.copy()
    else:
        iss = cache.copy()
    return iss

def extract_frs_items2():
    iss = get_issues()
    filtered = [i for i in iss.values() if i.sprint_milestone.name == "3.0"]
    reqs = []
    frs = set()
    for v in filtered:
        d = v.description or ""
        lines = d.split("\r\n")
        i = 0
        while i < len(lines):
            l = lines[i]
            m = magic.search(l)
            if m:
                web, fr = m.groups()
                web = web or ""
                rest = l[m.end():]
                other = []
                i += 1
                while i < len(lines):
                    l = lines[i]
                    if not l.strip():
                        break
                    elif magic.search(l):
                        break
                    else:
                        other.append(l.strip())
                    i += 1                        
                result = (v.id, web + fr, rest, "\n".join(other))
                reqs.append(result)
            i += 1    
    reqs.sort()
    return reqs

reqs= extract_frs_items2()
with open("test.txt", 'w') as f:
    for id, fr, rest, other in reqs:
        f.write("*%s*%s\n"%(fr, ": %s" % rest if rest else ""))
        f.write(other)
        f.write("\n\n")
import os; os.startfile(f.name)

In [4]:
from scripts.tools.issuetracker import IssuetrackerAPI
import re
try:
    cache
except NameError:
    cache = None
cache = cache or {}
magic = re.compile(r"[\+\*\s]*(3.0Web)?(FRS\d+[0-9\.]+)[\+\*: \-]*")

def get_issues():
    global cache
    if not cache:
        api = IssuetrackerAPI('issue.pbsbiotech.com', 'nstarkweather', 'kookychemist')
        iss = api.download_issues(project_id="pbssoftware")
        cache = iss
    else:
        iss = cache
    return [i for i in iss.values() if i.sprint_milestone.name == "3.0"]

def extract_frs_items3():
    filtered = get_issues()
    reqs = []
    frs = set()
    for v in filtered:
        d = v.description or ""
        if magic.search(d):
            reqs.append((v.id, d.strip().replace("\r\n", "\n")))
    reqs.sort()
    return reqs

reqs= extract_frs_items3()
with open("compiled_frs_descriptions.txt", 'w', newline="\r\n") as f:
    for id, desc in reqs:
        f.write("+*Issue %d:*+\n" % id)
        f.write(desc)
        f.write("\n\n")
#import os; os.startfile(f.name)

In [5]:
def extract_word_items(word="login", exclude=()):
    filtered = get_issues()
    exclude = frozenset(exclude)
    logins = []
    for v in filtered:
        if v.id in exclude: continue
        if magic.search(v.description) and word in v.description.lower():
            logins.append((v.id, v.description.strip().replace("\r\n", "\n")))
    logins.sort()
    return logins

exclude = 1073, 2828, 1759, 2367, 2445, 2878, 2881, 2911, 3077
logins = extract_word_items("login", exclude)
def write_word_items(word, items):
    with open("%s_issues.txt" % word, 'w') as f:
        for id, desc in items:
            f.write("+*Issue %d:*+\n" % id)
            f.write(desc)
            f.write("\n\n")
    return f.name
import os; os.startfile(write_word_items("login", logins))

In [6]:
def extract(word):
    import os; os.startfile(write_word_items(word, extract_word_items(word)))

In [14]:
import os; os.startfile(write_word_items("permission", extract_word_items("permission")))

In [8]:
extract("user events")

In [17]:
extract("midnight")